# 머신러닝 돌리기 (기본)
### history-stations
- 새로운 변수 추가, 튜닝 등 진행하지 않음

In [1]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 7.4 MB/s eta 0:00:00


#### 라이브러리 준비

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score
import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from prophet import Prophet
import holidays

from datetime import datetime, timedelta
now = datetime.now()

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


#### 방문자수 데이터 데이터 프레임 변환

In [3]:
from pymongo import MongoClient

client = MongoClient('주소 작성')

db = client.crawling

collection = db['주소 작성']

rows = collection.find()
history_stations = []
for row in rows:
    history_stations.append(row)

history_stations = pd.DataFrame(history_stations)

#### 머신러닝 코드 RandomForestRegressor

In [ ]:
def run_ml(i):
    df_h = pd.DataFrame(history_stations['history'][i]) # history_chargers의 i번째 history 가져오기

    # 변수 생성 (주말, 월, 일, 시간, 분) #########################
    df_h['weekday'] = df_h['time'].dt.weekday
    df_h['month'] = df_h['time'].dt.month
    df_h['day'] = df_h['time'].dt.day
    df_h['hour'] = df_h['time'].dt.hour
    df_h['minute'] = df_h['time'].dt.minute

    # 변수 생성 (공휴일)
    kr_holidays = holidays.KR()
    df_h['holiday'] = df_h.time.apply(lambda x: 1 if x in kr_holidays else 0)

    # ml 실행을 위해 날짜를 index로 설정하기
    df_h.set_index(keys='time', inplace=True)

    # count 변수 저장하기
    count = df_h.iloc[0, 0]

    # target(예측할 열) 설정하기
    target = 'visitNum'

    # x, y 값 설정하기
    x = df_h.drop(target, axis=1)
    y = df_h[target]

    # 데이터 샘플 수 확인 (10개 이하인 경우 예측 생략)
    if len(df_h) <= 10:
        return 0
    else:
        # 데이터가 10개 이상일 경우, 기존처럼 훈련/테스트 분할
        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=0)


    model = RandomForestRegressor(random_state=0)

    model.fit(x_train, y_train) # 모델 학습

    y_pred = model.predict(x_test) # 모델 예측
    y_pred = np.round(y_pred)

    print(history_stations['_id'][i], end=' ')
    print(r2_score(y_test, y_pred))  # 모델 정확도 출력
    acc = model.score(x_test, y_test)

    # 시각화를 위함 - df 형식으로 변환
    y_pred = pd.DataFrame(y_pred)
    y_test = pd.DataFrame(y_test)

    # 하나의 df로 합치기
    y_test.reset_index(inplace=True)
    df = pd.concat([y_test, y_pred], axis=1)
    df.columns = ['time', 'y_test', 'y_pred']

    # # 시각화
    # plt.figure(figsize=(8,6))
    # sns.lineplot(x='time' , y='y_test', data=df)
    # sns.lineplot(x='time' , y='y_pred', data=df)
    # plt.xticks(rotation=50)
    # plt.show()

    # 시간 단위별 예측 df 생성
    pre_df = pd.date_range(now.date()+ timedelta(days=1) , periods=24 , freq="30min") # 30분 단위로 예측 df 만들기

    pre_df = pd.DataFrame(pre_df) # 데이터 프레임 형태로 변환
    pre_df.columns=['time'] # 열 이름 변경

    pre_df['count'] = count

    # 변수 추가하기
    pre_df['time'] = pd.to_datetime(pre_df['time'])
    pre_df['weekday'] = pre_df['time'].dt.weekday
    pre_df['month'] = pre_df['time'].dt.month
    pre_df['day'] = pre_df['time'].dt.day
    pre_df['hour'] = pre_df['time'].dt.hour
    pre_df['minute'] = pre_df['time'].dt.minute

    kr_holidays = holidays.KR()
    pre_df['holiday'] = pre_df.time.apply(lambda x: 1 if x in kr_holidays else 0)

    pre_df.set_index(keys='time', inplace=True)

    # print(pre_df)
    pre_predict = model.predict(pre_df)
    pre_predict= np.round(pre_predict)
    # print(pre_predict)

    pre_predict = pre_predict.tolist()
    collection = db['demand-info']

    statId = history_stations['_id'][i]
    # 해당 statId를 가진 문서 조회
    existing_doc = collection.find_one({"statId": statId })

    # 문서가 존재하지 않으면 새로운 문서를 추가하고 업데이트
    if existing_doc is None:
        new_doc = { "statId": statId, "demandInfo": { "viewNum": 0, "departsIn30m": [], "hourlyVisitNum": [] } }
        result = collection.insert_one(new_doc)
        print("Added new document")
        #time.sleep(0.1)

    x = collection.update_one(
        {"statId":statId},
        {"$set" : {
            'demandInfo.hourlyVisitNum' : pre_predict
        }
        })

    return acc

- 5/23에 새로 생겨난 충전소에 대해서 데이터가 없어서 모델을 돌릴 수 없음 => 시간 경과 자동 해결

- blue = test, orange = predict

In [ ]:
arr = []
for i in range(len(history_stations)):
    arr.append(run_ml(i))
print(np.mean(arr))

AM900018 1.0
AM900036 -0.021479713603818507
AM900037 0.7424074074074074
AM900038 0.5296703296703296
BE241061 -0.5141509433962266
BNBN0067 0.8180640459900409
BNBN0111 0.5480030721966205
BNBN0121 0.741771730537178
BNBN0202 0.3238546603475515
BNBN0276 -1.009389671361502
BNBN0373 0.5910261736601579
BNBN0408 0.8888364185870797
BNBN0437 0.45938040154234816
BNBN0596 0.6942857142857144
BNBN0604 0.48058252427184467
BNLA0101 1.0
BNLA0301 -0.9478155339805823
BNLA0302 1.0
BNLA0303 0.15521869572082747
BNLA0304 -0.04390243902438984
BNLD1901 1.0
BNLE2901 1.0
BNLJ5401 0.4851643945469125
CG000011 0.24216138787203556
CGC00001 0.7555312055580164
CGC00002 0.8404473438956197
CHB00008 1.0
CI0G0001 -0.352290679304897
CPCP1018 0.7197619047619048
CPCP1019 0.5360433604336043
CSST0001 1.0
CU420001 0.6852941176470588
CU420002 1.0
CV000026 -0.25882352941176445
CV000212 0.6774983517001036
CV000249 0.8014934372246185
CV000264 1.0
CV000323 0.8249951895324226
CV000324 0.685986793837124
CV000330 1.0
CV000332 0.83929409

# 결정계수 r2 값이 0.7이상이면 좋은 모델, 0.3 이상이면 평범한 모델로 평가

In [ ]:
## NaN 값이 있는지 확인
has_none_or_nan = any(x is None or (isinstance(x, float) and np.isnan(x)) for x in arr)
print(has_none_or_nan) # nan값이 있으면 True

## NaN 값이 있는 행 확인
indices_with_none_or_nan = [i for i, x in enumerate(arr) if x is None or (isinstance(x, float) and np.isnan(x))]
print(indices_with_none_or_nan)

## NaN 값 제거
filtered_list = [x for x in arr if x is not None and not (isinstance(x, float) and np.isnan(x))]
print(filtered_list)

False
[]
[1.0, 0.045312291169451346, 0.8303217777777777, 0.5934211648351648, 0.1836202830188679, 0.817034808307356, 0.6426416410650281, 0.7608859651312778, 0.23680094786729877, -0.1604225352112676, 0.6922448248165074, 0.9117916981488478, 0.5373121393431726, 0.8066968571428572, 0.6737668689320389, 1.0, -0.16744271844660163, 1.0, 0.2973852518553609, 0.3294086720867211, 1.0, 1.0, 0.6407734562951082, 0.33358993315295227, 0, 0.8096002191499778, 0.8814962534948741, 1.0, -0.11536935229067913, 0.7798704285714286, 0.6467086178861788, 1.0, 0.7614300534759358, 1.0, 0.073736666666667, 0.6457178298954507, 0.8228775566995965, 1.0, 0.8544804117760246, 0.7669158473954512, 1.0, 0.8629680841077632, 1.0, 0.8315174191986442, 0.6482255324511657, 1.0, 1.0, 0.7702073293632616, 0.6800062199185486, 0.7682072317596567, 0.40842122641509426, 0.7246936571125264, 0.8531773913043478, 0.9835604368932038, 0.7772413580514914, 0.8469404991408012, 0.6562959349593496, 0.6758634443074394, 0.9698205128205128, 0.850457211538

In [ ]:
print(min(arr))
print(max(arr))
print(len(arr))
print(sum(filtered_list) / len(filtered_list))
# arr.index(0.9711363007518797)

-1.2895981132075476
1.0
2468
0.7486383765878828


#### 머신러닝 코드 LinearRegression

In [ ]:
def run_ml(i):
    df_h = pd.DataFrame(history_stations['history'][i]) # history_chargers의 i번째 history 가져오기

    # 변수 생성 (주말, 월, 일, 시간, 분) #########################
    df_h['weekday'] = df_h['time'].dt.weekday
    df_h['month'] = df_h['time'].dt.month
    df_h['day'] = df_h['time'].dt.day
    df_h['hour'] = df_h['time'].dt.hour
    df_h['minute'] = df_h['time'].dt.minute

    # 변수 생성 (공휴일)
    kr_holidays = holidays.KR()
    df_h['holiday'] = df_h.time.apply(lambda x: 1 if x in kr_holidays else 0)

    # ml 실행을 위해 날짜를 index로 설정하기
    df_h.set_index(keys='time', inplace=True)

    # count 변수 저장하기
    count = df_h.iloc[0, 0]

    # target(예측할 열) 설정하기
    target = 'visitNum'

    # x, y 값 설정하기
    x = df_h.drop(target, axis=1)
    y = df_h[target]

    # 데이터 샘플 수 확인 (10개 이하인 경우 예측 생략)
    if len(df_h) <= 10:
        return 0
    else:
        # 데이터가 10개 이상일 경우, 기존처럼 훈련/테스트 분할
        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=0)


    model = LinearRegression()

    model.fit(x_train, y_train) # 모델 학습

    y_pred = model.predict(x_test) # 모델 예측
    y_pred = np.round(y_pred)

    print(history_stations['_id'][i], end=' ')
    print(r2_score(y_test, y_pred))  # 모델 정확도 출력
    acc = model.score(x_test, y_test)

    # 시각화를 위함 - df 형식으로 변환
    y_pred = pd.DataFrame(y_pred)
    y_test = pd.DataFrame(y_test)

    # 하나의 df로 합치기
    y_test.reset_index(inplace=True)
    df = pd.concat([y_test, y_pred], axis=1)
    df.columns = ['time', 'y_test', 'y_pred']

    # # 시각화
    # plt.figure(figsize=(8,6))
    # sns.lineplot(x='time' , y='y_test', data=df)
    # sns.lineplot(x='time' , y='y_pred', data=df)
    # plt.xticks(rotation=50)
    # plt.show()

    # 시간 단위별 예측 df 생성
    pre_df = pd.date_range(now.date()+ timedelta(days=1) , periods=24 , freq="30min") # 30분 단위로 예측 df 만들기

    pre_df = pd.DataFrame(pre_df) # 데이터 프레임 형태로 변환
    pre_df.columns=['time'] # 열 이름 변경

    pre_df['count'] = count

    # 변수 추가하기
    pre_df['time'] = pd.to_datetime(pre_df['time'])
    pre_df['weekday'] = pre_df['time'].dt.weekday
    pre_df['month'] = pre_df['time'].dt.month
    pre_df['day'] = pre_df['time'].dt.day
    pre_df['hour'] = pre_df['time'].dt.hour
    pre_df['minute'] = pre_df['time'].dt.minute

    kr_holidays = holidays.KR()
    pre_df['holiday'] = pre_df.time.apply(lambda x: 1 if x in kr_holidays else 0)

    pre_df.set_index(keys='time', inplace=True)

    pre_predict = model.predict(pre_df)
    pre_predict= np.round(pre_predict)

    pre_predict = pre_predict.tolist()
    collection = db['demand-info']

    statId = history_stations['_id'][i]
    # 해당 statId를 가진 문서 조회
    existing_doc = collection.find_one({"statId": statId })

    # 문서가 존재하지 않으면 새로운 문서를 추가하고 업데이트
    if existing_doc is None:
        new_doc = { "statId": statId, "demandInfo": { "viewNum": 0, "departsIn30m": [], "hourlyVisitNum": [] } }
        result = collection.insert_one(new_doc)
        print("Added new document")
        #time.sleep(0.1)

    x = collection.update_one(
        {"statId":statId},
        {"$set" : {
            'demandInfo.hourlyVisitNum' : pre_predict
        }
        })

    return acc

- 5/23에 새로 생겨난 충전소에 대해서 데이터가 없어서 모델을 돌릴 수 없음 => 시간 경과 자동 해결

- blue = test, orange = predict

In [ ]:
arr = []
for i in range(len(history_stations)):
    arr.append(run_ml(i))
print(np.mean(arr))

AM900018 1.0
AM900036 -0.021479713603818507
AM900037 -0.228518518518519
AM900038 -0.17582417582417587
BE241061 -0.009433962264151052
BNBN0067 0.09898384680782157
BNBN0111 -0.53405017921147
BNBN0121 -0.0061998085965131455
BNBN0202 -0.0142180094786728
BNBN0276 -0.004694835680751019
BNBN0373 0.04572773854036838
BNBN0408 0.03995088779750655
BNBN0437 -0.22350751229889632
BNBN0596 -0.06999999999999984
BNBN0604 -0.03883495145631066
BNLA0101 1.0
BNLA0301 -0.03883495145631066
BNLA0302 1.0
BNLA0303 -0.11511132164850757
BNLA0304 -0.04390243902438984
BNLD1901 1.0
BNLE2901 1.0
BNLJ5401 -0.11547714514835627
CG000011 -0.4390418589845617
CGC00001 0.09696220828573421
CGC00002 -0.1567567567567567
CHB00008 1.0
CI0G0001 -0.0142180094786728
CPCP1018 -0.171904761904762
CPCP1019 -0.04390243902439006
CSST0001 1.0
CU420001 -0.28743315508021405
CU420002 1.0
CV000026 -0.04901960784313708
CV000212 -0.43110106433079043
CV000249 0.1364964519270906
CV000264 1.0
CV000323 -0.07576486434481433
CV000324 -0.1775495231107

# 결정계수 r2 값이 0.7이상이면 좋은 모델, 0.3 이상이면 평범한 모델로 평가

In [ ]:
## NaN 값이 있는지 확인
has_none_or_nan = any(x is None or (isinstance(x, float) and np.isnan(x)) for x in arr)
print(has_none_or_nan) # nan값이 있으면 True

## NaN 값이 있는 행 확인
indices_with_none_or_nan = [i for i, x in enumerate(arr) if x is None or (isinstance(x, float) and np.isnan(x))]
print(indices_with_none_or_nan)

## NaN 값 제거
filtered_list = [x for x in arr if x is not None and not (isinstance(x, float) and np.isnan(x))]
print(filtered_list)

False
[]
[1.0, 0.045312291169451346, 0.8303217777777777, 0.5934211648351648, 0.1836202830188679, 0.817034808307356, 0.6426416410650281, 0.7608859651312778, 0.23680094786729877, -0.1604225352112676, 0.6922448248165074, 0.9117916981488478, 0.5373121393431726, 0.8066968571428572, 0.6737668689320389, 1.0, -0.16744271844660163, 1.0, 0.2973852518553609, 0.3294086720867211, 1.0, 1.0, 0.6407734562951082, 0.33358993315295227, 0, 0.8096002191499778, 0.8814962534948741, 1.0, -0.11536935229067913, 0.7798704285714286, 0.6467086178861788, 1.0, 0.7614300534759358, 1.0, 0.073736666666667, 0.6457178298954507, 0.8228775566995965, 1.0, 0.8544804117760246, 0.7669158473954512, 1.0, 0.8629680841077632, 1.0, 0.8315174191986442, 0.6482255324511657, 1.0, 1.0, 0.7702073293632616, 0.6800062199185486, 0.7682072317596567, 0.40842122641509426, 0.7246936571125264, 0.8531773913043478, 0.9835604368932038, 0.7772413580514914, 0.8469404991408012, 0.6562959349593496, 0.6758634443074394, 0.9698205128205128, 0.850457211538

In [ ]:
print(min(arr))
print(max(arr))
print(len(arr))
print(sum(filtered_list) / len(filtered_list))
# arr.index(0.9711363007518797)

-1.2895981132075476
1.0
2468
0.7486383765878828


#### 머신러닝 코드 LGBMRegressor

In [ ]:
def run_ml(i):
    df_h = pd.DataFrame(history_stations['history'][i]) # history_chargers의 i번째 history 가져오기

    # 변수 생성 (주말, 월, 일, 시간, 분) #########################
    df_h['weekday'] = df_h['time'].dt.weekday
    df_h['month'] = df_h['time'].dt.month
    df_h['day'] = df_h['time'].dt.day
    df_h['hour'] = df_h['time'].dt.hour
    df_h['minute'] = df_h['time'].dt.minute

    # 변수 생성 (공휴일)
    kr_holidays = holidays.KR()
    df_h['holiday'] = df_h.time.apply(lambda x: 1 if x in kr_holidays else 0)

    # ml 실행을 위해 날짜를 index로 설정하기
    df_h.set_index(keys='time', inplace=True)

    # count 변수 저장하기
    count = df_h.iloc[0, 0]

    # target(예측할 열) 설정하기
    target = 'visitNum'

    # x, y 값 설정하기
    x = df_h.drop(target, axis=1)
    y = df_h[target]

    # 데이터 샘플 수 확인 (10개 이하인 경우 예측 생략)
    if len(df_h) <= 10:
        return 0
    else:
        # 데이터가 10개 이상일 경우, 기존처럼 훈련/테스트 분할
        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=0)


    model = LGBMRegressor(random_state=0, verbose=-1)

    model.fit(x_train, y_train) # 모델 학습

    y_pred = model.predict(x_test) # 모델 예측
    y_pred = np.round(y_pred)

    print(history_stations['_id'][i], end=' ')
    print(r2_score(y_test, y_pred))  # 모델 정확도 출력
    acc = model.score(x_test, y_test)

    # 시각화를 위함 - df 형식으로 변환
    y_pred = pd.DataFrame(y_pred)
    y_test = pd.DataFrame(y_test)

    # 하나의 df로 합치기
    y_test.reset_index(inplace=True)
    df = pd.concat([y_test, y_pred], axis=1)
    df.columns = ['time', 'y_test', 'y_pred']

    # # 시각화
    # plt.figure(figsize=(8,6))
    # sns.lineplot(x='time' , y='y_test', data=df)
    # sns.lineplot(x='time' , y='y_pred', data=df)
    # plt.xticks(rotation=50)
    # plt.show()

    # 시간 단위별 예측 df 생성
    pre_df = pd.date_range(now.date()+ timedelta(days=1) , periods=24 , freq="30min") # 30분 단위로 예측 df 만들기

    pre_df = pd.DataFrame(pre_df) # 데이터 프레임 형태로 변환
    pre_df.columns=['time'] # 열 이름 변경

    pre_df['count'] = count

    # 변수 추가하기
    pre_df['time'] = pd.to_datetime(pre_df['time'])
    pre_df['weekday'] = pre_df['time'].dt.weekday
    pre_df['month'] = pre_df['time'].dt.month
    pre_df['day'] = pre_df['time'].dt.day
    pre_df['hour'] = pre_df['time'].dt.hour
    pre_df['minute'] = pre_df['time'].dt.minute

    kr_holidays = holidays.KR()
    pre_df['holiday'] = pre_df.time.apply(lambda x: 1 if x in kr_holidays else 0)

    pre_df.set_index(keys='time', inplace=True)

    pre_predict = model.predict(pre_df)
    pre_predict= np.round(pre_predict)

    pre_predict = pre_predict.tolist()
    collection = db['demand-info']

    statId = history_stations['_id'][i]
    # 해당 statId를 가진 문서 조회
    existing_doc = collection.find_one({"statId": statId })

    # 문서가 존재하지 않으면 새로운 문서를 추가하고 업데이트
    if existing_doc is None:
        new_doc = { "statId": statId, "demandInfo": { "viewNum": 0, "departsIn30m": [], "hourlyVisitNum": [] } }
        result = collection.insert_one(new_doc)
        print("Added new document")
        #time.sleep(0.1)

    x = collection.update_one(
        {"statId":statId},
        {"$set" : {
            'demandInfo.hourlyVisitNum' : pre_predict
        }
        })

    return acc

- 5/23에 새로 생겨난 충전소에 대해서 데이터가 없어서 모델을 돌릴 수 없음 => 시간 경과 자동 해결

- blue = test, orange = predict

In [ ]:
arr = []
for i in range(len(history_stations)):
    arr.append(run_ml(i))
print(np.mean(arr))

AM900018 1.0
AM900036 -0.021479713603818507
AM900037 0.6928703703703702
AM900038 0.36505494505494496
BE241061 -0.009433962264151052
BNBN0067 0.740091494271487
BNBN0111 0.5206093189964156
BNBN0121 0.7239628843673283
BNBN0202 -0.0142180094786728
BNBN0276 1.0
BNBN0373 0.49619166320454233
BNBN0408 0.8787306384586324
BNBN0437 0.4024730753889112
BNBN0596 0.6942857142857144
BNBN0604 0.48058252427184467
BNLA0101 1.0
BNLA0301 -0.6881067961165046
BNLA0302 1.0
BNLA0303 0.25659245223432825
BNLA0304 0.07208672086720902
BNLD1901 1.0
BNLE2901 1.0
BNLJ5401 0.44226142742582186
CG000011 0.1484959414292535
CGC00001 0.6607371832233697
CGC00002 0.8005591798695247
CHB00008 1.0
CI0G0001 -0.0142180094786728
CPCP1018 0.6178571428571429
CPCP1019 0.7680216802168022
CSST0001 1.0
CU420001 0.8283422459893048
CU420002 1.0
CV000026 -0.04901960784313708
CV000212 0.3751530564189507
CV000249 0.741941468392004
CV000264 1.0
CV000323 0.7014623821435444
CV000324 0.607483492296405
CV000330 1.0
CV000332 0.7991176194499202
CV0

# 결정계수 r2 값이 0.7이상이면 좋은 모델, 0.3 이상이면 평범한 모델로 평가

In [ ]:
## NaN 값이 있는지 확인
has_none_or_nan = any(x is None or (isinstance(x, float) and np.isnan(x)) for x in arr)
print(has_none_or_nan) # nan값이 있으면 True

## NaN 값이 있는 행 확인
indices_with_none_or_nan = [i for i, x in enumerate(arr) if x is None or (isinstance(x, float) and np.isnan(x))]
print(indices_with_none_or_nan)

## NaN 값 제거
filtered_list = [x for x in arr if x is not None and not (isinstance(x, float) and np.isnan(x))]
print(filtered_list)

False
[]
[1.0, 0.045312291169451346, 0.8303217777777777, 0.5934211648351648, 0.1836202830188679, 0.817034808307356, 0.6426416410650281, 0.7608859651312778, 0.23680094786729877, -0.1604225352112676, 0.6922448248165074, 0.9117916981488478, 0.5373121393431726, 0.8066968571428572, 0.6737668689320389, 1.0, -0.16744271844660163, 1.0, 0.2973852518553609, 0.3294086720867211, 1.0, 1.0, 0.6407734562951082, 0.33358993315295227, 0, 0.8096002191499778, 0.8814962534948741, 1.0, -0.11536935229067913, 0.7798704285714286, 0.6467086178861788, 1.0, 0.7614300534759358, 1.0, 0.073736666666667, 0.6457178298954507, 0.8228775566995965, 1.0, 0.8544804117760246, 0.7669158473954512, 1.0, 0.8629680841077632, 1.0, 0.8315174191986442, 0.6482255324511657, 1.0, 1.0, 0.7702073293632616, 0.6800062199185486, 0.7682072317596567, 0.40842122641509426, 0.7246936571125264, 0.8531773913043478, 0.9835604368932038, 0.7772413580514914, 0.8469404991408012, 0.6562959349593496, 0.6758634443074394, 0.9698205128205128, 0.850457211538

In [ ]:
print(min(arr))
print(max(arr))
print(len(arr))
print(sum(filtered_list) / len(filtered_list))
# arr.index(0.9711363007518797)

-1.2895981132075476
1.0
2468
0.7486383765878828


#### 머신러닝 코드 xgboost

In [10]:
def run_ml(i):
    df_h = pd.DataFrame(history_stations['history'][i]) # history_chargers의 i번째 history 가져오기

    # 변수 생성 (주말, 월, 일, 시간, 분) #########################
    df_h['weekday'] = df_h['time'].dt.weekday
    df_h['month'] = df_h['time'].dt.month
    df_h['day'] = df_h['time'].dt.day
    df_h['hour'] = df_h['time'].dt.hour
    df_h['minute'] = df_h['time'].dt.minute

    # 변수 생성 (공휴일)
    kr_holidays = holidays.KR()
    df_h['holiday'] = df_h.time.apply(lambda x: 1 if x in kr_holidays else 0)

    # ml 실행을 위해 날짜를 index로 설정하기
    df_h.set_index(keys='time', inplace=True)

    # count 변수 저장하기
    count = df_h.iloc[0, 0]

    # target(예측할 열) 설정하기
    target = 'visitNum'

    # x, y 값 설정하기
    x = df_h.drop(target, axis=1)
    y = df_h[target]

    # 데이터 샘플 수 확인 (10개 이하인 경우 예측 생략)
    if len(df_h) <= 10:
        return 0
    else:
        # 데이터가 10개 이상일 경우, 기존처럼 훈련/테스트 분할
        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=0)


    model = xgb.XGBRegressor(
        eval_metric='rmse'  # 회귀에서는 r2를 사용하는 경우
    )

    model.fit(x_train, y_train) # 모델 학습

    y_pred = model.predict(x_test) # 모델 예측
    y_pred = np.round(y_pred)

    print(history_stations['_id'][i], end=' ')
    print(r2_score(y_test, y_pred))  # 모델 정확도 출력
    acc = model.score(x_test, y_test)

    # 시각화를 위함 - df 형식으로 변환
    y_pred = pd.DataFrame(y_pred)
    y_test = pd.DataFrame(y_test)

    # 하나의 df로 합치기
    y_test.reset_index(inplace=True)
    df = pd.concat([y_test, y_pred], axis=1)
    df.columns = ['time', 'y_test', 'y_pred']

    # # 시각화
    # plt.figure(figsize=(8,6))
    # sns.lineplot(x='time' , y='y_test', data=df)
    # sns.lineplot(x='time' , y='y_pred', data=df)
    # plt.xticks(rotation=50)
    # plt.show()

    # 시간 단위별 예측 df 생성
    pre_df = pd.date_range(now.date()+ timedelta(days=1) , periods=24 , freq="30min") # 30분 단위로 예측 df 만들기

    pre_df = pd.DataFrame(pre_df) # 데이터 프레임 형태로 변환
    pre_df.columns=['time'] # 열 이름 변경

    pre_df['count'] = count

    # 변수 추가하기
    pre_df['time'] = pd.to_datetime(pre_df['time'])
    pre_df['weekday'] = pre_df['time'].dt.weekday
    pre_df['month'] = pre_df['time'].dt.month
    pre_df['day'] = pre_df['time'].dt.day
    pre_df['hour'] = pre_df['time'].dt.hour
    pre_df['minute'] = pre_df['time'].dt.minute

    kr_holidays = holidays.KR()
    pre_df['holiday'] = pre_df.time.apply(lambda x: 1 if x in kr_holidays else 0)

    pre_df.set_index(keys='time', inplace=True)

    pre_predict = model.predict(pre_df)
    pre_predict= np.round(pre_predict)

    pre_predict = pre_predict.tolist()
    collection = db['demand-info']

    statId = history_stations['_id'][i]
    # 해당 statId를 가진 문서 조회
    existing_doc = collection.find_one({"statId": statId })

    # 문서가 존재하지 않으면 새로운 문서를 추가하고 업데이트
    if existing_doc is None:
        new_doc = { "statId": statId, "demandInfo": { "viewNum": 0, "departsIn30m": [], "hourlyVisitNum": [] } }
        result = collection.insert_one(new_doc)
        print("Added new document")
        #time.sleep(0.1)

    x = collection.update_one(
        {"statId":statId},
        {"$set" : {
            'demandInfo.hourlyVisitNum' : pre_predict
        }
        })

    return acc

In [11]:
arr = []
for i in range(len(history_stations)):
    arr.append(run_ml(i))
print(np.mean(arr))

AM900018 1.0
AM900036 0.025547564029693604
AM900037 0.7915148138999939
AM900038 0.03210771083831787
BE241061 -1.696969985961914
BNBN0067 0.914193868637085
BNBN0111 0.3984346389770508
BNBN0121 0.6690219640731812
BNBN0202 1.0
BNBN0276 0.0
BNBN0373 0.4887181520462036
BNBN0408 0.8693658113479614
BNBN0437 0.53493332862854
BNBN0596 0.646942138671875
BNBN0604 0.5829986333847046
BNLA0101 1.0
BNLA0301 -0.9329150915145874
BNLA0302 1.0
BNLA0303 -0.15995275974273682
BNLA0304 -0.030888080596923828
BNLD1901 1.0
BNLE2901 1.0
BNLJ5401 -0.20799696445465088
CG000011 0.00224226713180542
CG000060 1.0
CGC00001 0.7813801765441895
CGC00002 0.8159596920013428
CHB00008 1.0
CI0G0001 0.11978024244308472
CPCP1018 0.6434929370880127
CPCP1019 1.0
CSST0001 1.0
CU420001 0.6190067529678345
CU420002 1.0
CV000026 -0.3568603992462158
CV000212 0.41300255060195923
CV000249 0.8182231187820435
CV000264 1.0
CV000323 0.7273575067520142
CV000324 0.4525323510169983
CV000330 1.0
CV000332 0.8476607203483582
CV000333 1.0
CV000360 0

# 결정계수 r2 값이 0.7이상이면 좋은 모델, 0.3 이상이면 평범한 모델로 평가

In [12]:
## NaN 값이 있는지 확인
has_none_or_nan = any(x is None or (isinstance(x, float) and np.isnan(x)) for x in arr)
print(has_none_or_nan) # nan값이 있으면 True

## NaN 값이 있는 행 확인
indices_with_none_or_nan = [i for i, x in enumerate(arr) if x is None or (isinstance(x, float) and np.isnan(x))]
print(indices_with_none_or_nan)

## NaN 값 제거
filtered_list = [x for x in arr if x is not None and not (isinstance(x, float) and np.isnan(x))]
print(filtered_list)

False
[]
[1.0, 0.24255478382110596, 0.8212380409240723, 0.2148621678352356, -0.9291990995407104, 0.9091524481773376, 0.48572778701782227, 0.6974716782569885, 0.0, 0.0, 0.5095657706260681, 0.8559607863426208, 0.5412232875823975, 0.6448016166687012, 0.5863996744155884, 1.0, -0.9430555105209351, 1.0, 0.008086860179901123, 0.11590379476547241, 1.0, 1.0, 0.09408760070800781, 0.12188112735748291, 1.0, 0.8276888728141785, 0.8672562837600708, 1.0, -0.15264177322387695, 0.6918106079101562, 0.9990991353988647, 1.0, 0.6962590217590332, 1.0, -0.3094898462295532, 0.48485004901885986, 0.8544051647186279, 1.0, 0.7606188654899597, 0.5106449127197266, 1.0, 0.8193660974502563, 1.0, 0.8653199076652527, 0.7261618375778198, 1.0, 1.0, 0.6321791410446167, 0.41334235668182373, 0.7495466470718384, 0.24282950162887573, 0.7575448155403137, 0.805411696434021, 0.5235550403594971, 0.7678497433662415, 0.7713685631752014, 0.611638069152832, 0.5194895267486572, 0.999500036239624, 0.7533179521560669, 0.1563127636909485

In [13]:
print(min(arr))
print(max(arr))
print(len(arr))
print(sum(filtered_list) / len(filtered_list))
# arr.index(0.9711363007518797)

-6.414550304412842
1.0
2468
0.6801715043752274


# 머신러닝 성능 테스트 ⭐
(초기 0.65)
## 평가 기준 r2(결정계수)
## 1. RandomForestRegressor 👍👍
## 결과: 0.7486
## 2. LinearRegression
## 결과: 0.2363
## 3. LGBMRegressor
## 결과: 0.7023
## 4. xgboost
## 결과: 0.6801